# Multi-tube design example (simple)

Design a strand displacement gate. 

See the accompanying design specification (PDF file). See also the LaTeX spec file that you can edit to make your own design specs in a standardized format.

This is a 1-step reaction. To design 1 gate, there are 2 elementary step tubes plus 1 global crosstalk tube. 

Target test tubes: 
- Reactants (Step 0)
- Products (Step 1) 
- Global crosstalk

Material: RNA    
Temperature: 23 C

In [1]:
# Import Python NUPACK module
from nupack import *

In [2]:
# Define physical model
my_model = Model(material='rna', celsius=23)

# Define sequence domains
da = Domain('N10', name='da')
db = Domain('N8', name='db')

# Define strands containing these domains
sX = TargetStrand([da, db], name='sX')
sA = TargetStrand([~db, ~da], name='sA')
sB = TargetStrand([da], name='sB')  # ~dgate is the reverse complement of dgate
sA_toe = TargetStrand([~db], name='sA_toe')

# Define target complexes
cX = TargetComplex([sX], 'U18', name='cX')
cAB = TargetComplex([sA, sB], 'U8D10+', name='cAB')
cXA = TargetComplex([sX, sA], 'D18+', name='cXA')
cB = TargetComplex([sB], 'U10', name='cB')
cA_toe = TargetComplex([sA_toe], 'U8', name='cA_toe')

# Define target test tubes 
reactants = TargetTube(on_targets={cX: 1e-08, cAB: 1e-08}, 
                       off_targets=SetSpec(max_size=2, exclude=[cXA]), name='reactants')
products = TargetTube(on_targets={cXA: 1e-08, cB: 1e-08}, 
                      off_targets=SetSpec(max_size=2), name='products')
crosstalk = TargetTube(on_targets={cX: 1e-08, cAB: 1e-08, cA_toe: 1e-08, cB:1e-8}, 
                       off_targets=SetSpec(max_size=2, exclude=[cXA, [sX, sA_toe]]), name='crosstalk')

# Set a stop condition of 2%
# Set seed for random number generation to get a reproducible result for this demo
my_options = DesignOptions(f_stop=0.02, seed=93)

# Define and run the test tube design job
my_design = tube_design(tubes=[reactants, products, crosstalk], model=my_model, options=my_options)
my_results = my_design.run(trials=1)[0]

# Display the design results
my_results

Domain,Sequence
da,CAGGUGGAAG
da*,CUUCCACCUG
db,AGGUAAGC
db*,GCUUACCU
Strand,Sequence
sA_toe,GCUUACCU
sX,CAGGUGGAAGAGGUAAGC
sA,GCUUACCUCUUCCACCUG
sB,CAGGUGGAAG
Objective type,Value
